In [16]:
#Importar o primeiro banco de dados de leitos
import pandas as pd
leitos_23 = pd.read_csv("Leitos_2023.csv", encoding="latin-1")
display(leitos_23.info())







<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84471 entries, 0 to 84470
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   COMP                    84471 non-null  int64  
 1   REGIAO                  84471 non-null  object 
 2   UF                      84471 non-null  object 
 3   MUNICIPIO               84471 non-null  object 
 4   MOTIVO_DESABILITACAO    0 non-null      float64
 5   CNES                    84471 non-null  int64  
 6   NOME_ESTABELECIMENTO    84471 non-null  object 
 7   RAZAO_SOCIAL            84471 non-null  object 
 8   TP_GESTAO               84471 non-null  object 
 9   CO_TIPO_UNIDADE         84471 non-null  int64  
 10  DS_TIPO_UNIDADE         84471 non-null  object 
 11  NATUREZA_JURIDICA       84471 non-null  int64  
 12  DESC_NATUREZA_JURIDICA  84471 non-null  object 
 13  NO_LOGRADOURO           84471 non-null  object 
 14  NU_ENDERECO             84471 non-null

None

In [17]:
leitos_24 = pd.read_csv("Leitos_2024.csv", encoding="latin-1")
display(leitos_24.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85225 entries, 0 to 85224
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   COMP                    85225 non-null  int64  
 1   REGIAO                  85225 non-null  object 
 2   UF                      85225 non-null  object 
 3   MUNICIPIO               85225 non-null  object 
 4   MOTIVO_DESABILITACAO    0 non-null      float64
 5   CNES                    85225 non-null  int64  
 6   NOME_ESTABELECIMENTO    85225 non-null  object 
 7   RAZAO_SOCIAL            85225 non-null  object 
 8   TP_GESTAO               85225 non-null  object 
 9   CO_TIPO_UNIDADE         85225 non-null  int64  
 10  DS_TIPO_UNIDADE         85225 non-null  object 
 11  NATUREZA_JURIDICA       85225 non-null  int64  
 12  DESC_NATUREZA_JURIDICA  85225 non-null  object 
 13  NO_LOGRADOURO           85225 non-null  object 
 14  NU_ENDERECO             85225 non-null

None

In [18]:
leitos_25 = pd.read_csv("Leitos_2025.csv", encoding="latin-1" , sep=";")
display(leitos_25.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50174 entries, 0 to 50173
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   COMP                    50174 non-null  int64  
 1   REGIAO                  50174 non-null  object 
 2   UF                      50174 non-null  object 
 3   CO_IBGE                 50174 non-null  int64  
 4   MUNICIPIO               50174 non-null  object 
 5   MOTIVO_DESABILITACAO    0 non-null      float64
 6   CNES                    50174 non-null  int64  
 7   NOME_ESTABELECIMENTO    50172 non-null  object 
 8   RAZAO_SOCIAL            50174 non-null  object 
 9   TP_GESTAO               50174 non-null  object 
 10  CO_TIPO_UNIDADE         50174 non-null  int64  
 11  DS_TIPO_UNIDADE         50174 non-null  object 
 12  NATUREZA_JURIDICA       50174 non-null  int64  
 13  DESC_NATUREZA_JURIDICA  50174 non-null  object 
 14  NO_LOGRADOURO           50174 non-null

None